In [1]:
# Info:
    # Script name: 0_create_annotation_references_simplified.ipynb
    # Purpose: Create annotation references for DMR analysis
    # Author: Meghan M. Sleeper
    # Input: gencode.v44.chr_patch_hapl_scaff.basic.annotation.gff3
    # Outputs: 
        # gencode.v44.annotation.genes.txt
        # gencode.v44.annotation.genes.txt.sorted.formatted.bed 
        # gencode.v44.annotation.genes.txt.sorted.formatted.bed.gz
        # gencode.v44.annotation.all.txt
        # gencode.v44.annotation.all.txt.sorted.formatted.bed 
        # gencode.v44.annotation.all.txt.sorted.formatted.bed.gz

# Notes:
    # Modified from: https://medium.com/intothegenomics/annotate-genes-and-genomic-coordinates-using-python-9259efa6ffc2

# Reminders to self:
    # gencode_exons_codons_utrs_cds.head(10)
    # gencode.feature.unique()
    # gencode.head()
    # gencode[gencode.feature == 'gene'].head(10)
    # list(gencode.attribute)[0:10]
    # gencode[gencode.attribute.str.contains('gene_type')].head(10)
    # magic commands: 
        # %reset can remove variables from memory

# Import packages
import pandas as pd
import numpy as np
import pysam

# Assign file paths
# unzipped gttf annotation reference
gencode_annotation_file = "/Users/meghansleeper/Desktop/farm-files/dmr-workflow/supplementary/gencode.v44.chr_patch_hapl_scaff.basic.annotation.gff3"

In [30]:
# %%bash -s gencode_annotation_file 

# # add code to unzip gencode_annotation_file if it is a .gz file
# if gencode_annotation_file.endswith(".gz"):
#     with gzip.open(gencode_annotation_file, 'rb') as f_in:
#         with open(gencode_annotation_file[:-3], 'wb') as f_out:
#             shutil.copyfileobj(f_in, f_out)

gunzip: /Users/meghansleeper/Desktop/farm-files/dmr-workflow/supplementary/gencode.v44.chr_patch_hapl_scaff.basic.annotation.gff3: unknown suffix -- ignored


In [2]:
# Create dataframe from gencode gtf annotation file
gencode = pd.read_table(gencode_annotation_file, 
                        comment="#", 
                        sep = "\t", 
                        names = ['seqname', 'source', 'feature', 
                                'start' , 'end', 'score', 'strand', 
                                'frame', 'attribute'
                                ]
                        )

In [3]:
# creating dataframes for grouped features that share the same attribute information to make parsing attributes easier
  # see https://www.gencodegenes.org/pages/data_format.html for attributes associated with each feature
    # gencode.feature.unique() returns:
        #    'gene', 'transcript', 'exon', 'CDS', 
        #    'start_codon', 'stop_codon',
        #    'five_prime_UTR', 'three_prime_UTR',
        #    'stop_codon_redefined_as_selenocysteine'

gencode_genes = gencode[(gencode.feature == "gene")][
    ['seqname', 'start', 'end', 'attribute','feature', 
     'strand', 'source']].copy().reset_index().drop('index', axis=1)

gencode_exons_codons_utrs_cds = gencode[
    (gencode.feature == "exon") | 
    (gencode.feature == "start_codon") | 
    (gencode.feature == "stop_codon") | 
    (gencode.feature == "five_prime_UTR") | 
    (gencode.feature == "three_prime_UTR") | 
    (gencode.feature == "CDS")
    ][['seqname', 'start', 'end', 'attribute', 'feature', 'strand', 'source']
      ].copy().reset_index().drop('index', axis=1)

gencode_transcripts_selenocyteines = gencode[
    (gencode.feature == "transcript") | 
    (gencode.feature == "stop_codon_redefined_as_selenocysteine")
    ][['seqname', 'start', 'end', 'attribute', 'feature', 'strand', 'source']
      ].copy().reset_index().drop('index', axis=1)

In [14]:
list(gencode.attribute)[100:150]

['ID=ENST00000466557.6;Parent=ENSG00000241860.7;gene_id=ENSG00000241860.7;transcript_id=ENST00000466557.6;gene_type=lncRNA;gene_name=ENSG00000241860;transcript_type=lncRNA;transcript_name=ENST00000466557;level=2;transcript_support_level=5;tag=not_best_in_genome_evidence,basic,Ensembl_canonical;havana_gene=OTTHUMG00000002480.4;havana_transcript=OTTHUMT00000007037.2',
 'ID=exon:ENST00000466557.6:1;Parent=ENST00000466557.6;gene_id=ENSG00000241860.7;transcript_id=ENST00000466557.6;gene_type=lncRNA;gene_name=ENSG00000241860;transcript_type=lncRNA;transcript_name=ENST00000466557;exon_number=1;exon_id=ENSE00001947154.2;level=2;transcript_support_level=5;tag=not_best_in_genome_evidence,basic,Ensembl_canonical;havana_gene=OTTHUMG00000002480.4;havana_transcript=OTTHUMT00000007037.2',
 'ID=exon:ENST00000466557.6:2;Parent=ENST00000466557.6;gene_id=ENSG00000241860.7;transcript_id=ENST00000466557.6;gene_type=lncRNA;gene_name=ENSG00000241860;transcript_type=lncRNA;transcript_name=ENST00000466557;exon

In [5]:
# functions for parsing gtf files based on format of attribute column

def gene_info(x):
# Extract gene names, gene id gene_type, level, and gene_id for each gene
# Use this function when targetting info on genes
    g_name = list(filter(lambda x: 'gene_name' in x,  x.split(";")))[0].split("=")[1]
    g_id = list(filter(lambda x: 'gene_id' in x,  x.split(";")))[0].split("=")[1]
    g_type = list(filter(lambda x: 'gene_type' in x,  x.split(";")))[0].split("=")[1]
    g_level = int(list(filter(lambda x: 'level' in x,  x.split(";")))[0].split("=")[1])
    return (g_name, g_id, g_type, g_level)

def all_info(x):
# Extract gene names, gene id, gene_type, level, exon_id, exon_number, transcript_name, and transcript_type for each exon
# Use this function when targetting info on exons, CDS, start_codon, stop_codon, five_prime_UTR, and three_prime_UTR
    g_name = list(filter(lambda x: 'gene_name' in x,  x.split(";")))[0].split("=")[1]
    g_id = list(filter(lambda x: 'gene_id' in x,  x.split(";")))[0].split("=")[1]
    g_type = list(filter(lambda x: 'gene_type' in x,  x.split(";")))[0].split("=")[1]
    g_level = int(list(filter(lambda x: 'level' in x,  x.split(";")))[0].split("=")[1])
    exon_id = list(filter(lambda x: 'exon_id' in x,  x.split(";")))[0].split("=")[1]
    exon_number = int(list(filter(lambda x: 'exon_number' in x,  x.split(";")))[0].split("=")[1])
    transcript_name = list(filter(lambda x: 'transcript_name' in x,  x.split(";")))[0].split("=")[1]
    transcript_type = list(filter(lambda x: 'transcript_type' in x,  x.split(";")))[0].split("=")[1]
    return (g_name, g_id, g_type, g_level, exon_id, exon_number, transcript_name, transcript_type)
    
def transcript_info(x):
# Extract gene names, gene id, gene_type, level, transcript_id, and transcript_type for each transcript
# Use this function when targetting info on transcripts and stop_codon_redefined_as_selenocysteine
    g_name = list(filter(lambda x: 'gene_name' in x,  x.split(";")))[0].split("=")[1]
    g_id = list(filter(lambda x: 'gene_id' in x,  x.split(";")))[0].split("=")[1]
    g_type = list(filter(lambda x: 'gene_type' in x,  x.split(";")))[0].split("=")[1]
    g_level = int(list(filter(lambda x: 'level' in x,  x.split(";")))[0].split("=")[1])
    transcript_name = list(filter(lambda x: 'transcript_name' in x,  x.split(";")))[0].split("=")[1]
    transcript_type = list(filter(lambda x: 'transcript_type' in x,  x.split(";")))[0].split("=")[1]
    return (g_name, g_id, g_type, g_level, transcript_name, transcript_type)

In [15]:
## Using gene_info function on genes
# extract gene_name, gene_id, gene_type, and gene_level from gencode_genes.attribute
gencode_genes["gene_name"], gencode_genes["gene_id"], gencode_genes["gene_type"], gencode_genes["gene_level"] = zip(*gencode_genes.attribute.apply(lambda x: gene_info(x)))

## Using all_info function on exons, start codons, stop codons, utrs, and cds
# extract gene_name, gene_type, gene_level, exon_id, exon_number, transcript_name, and transcript_type from gencode_exons_codons_utrs_cds.attribute
gencode_exons_codons_utrs_cds["gene_name"], gencode_exons_codons_utrs_cds["gene_id"], gencode_exons_codons_utrs_cds["gene_type"], gencode_exons_codons_utrs_cds["gene_level"], gencode_exons_codons_utrs_cds["exon_id"], gencode_exons_codons_utrs_cds["exon_number"], gencode_exons_codons_utrs_cds["transcript_name"], gencode_exons_codons_utrs_cds["transcript_type"] = zip(*gencode_exons_codons_utrs_cds.attribute.apply(lambda x: all_info(x)))

## Using transcript_info function on transcripts and stop_codon_redefined_as_selenocysteine
# extract gene_name, gene_type, gene_level, exon_id, exon_number, transcript_name, and transcript_type from gencode_transcripts_selenocyteines.attribute using transcript_info function
gencode_transcripts_selenocyteines["gene_name"], gencode_transcripts_selenocyteines["gene_id"], gencode_transcripts_selenocyteines["gene_type"], gencode_transcripts_selenocyteines["gene_level"], gencode_transcripts_selenocyteines["transcript_name"], gencode_transcripts_selenocyteines["transcript_type"] = zip(*gencode_transcripts_selenocyteines.attribute.apply(lambda x: transcript_info(x)))

In [16]:
# gencode_genes.gene_type.unique() returns:
    # array(['lncRNA', 'transcribed_unprocessed_pseudogene',
    #        'unprocessed_pseudogene', 'miRNA', 'protein_coding',
    #        'processed_pseudogene', 'snRNA',
    #        'transcribed_processed_pseudogene', 'misc_RNA', 'TEC',
    #        'transcribed_unitary_pseudogene', 'snoRNA', 'scaRNA',
    #        'rRNA_pseudogene', 'unitary_pseudogene', 'pseudogene', 'rRNA',
    #        'IG_V_pseudogene', 'scRNA', 'IG_V_gene', 'IG_C_gene', 'IG_J_gene',
    #        'sRNA', 'ribozyme', 'translated_processed_pseudogene', 'vault_RNA',
    #        'TR_C_gene', 'TR_J_gene', 'TR_V_gene', 'TR_V_pseudogene',
    #        'TR_D_gene', 'IG_C_pseudogene', 'TR_J_pseudogene',
    #        'IG_J_pseudogene', 'IG_D_gene', 'IG_pseudogene', 'artifact',
    #        'Mt_tRNA', 'Mt_rRNA'], dtype=object)

# For filtering out the genes that are protein coding into a new dataframe, uncomment the following line
# gencode_genes_protein_coding = gencode_genes[gencode_genes['gene_type'] == 'protein_coding'].reset_index().drop('index', axis=1)

# concatenating dataframes for all features into one datafram with NaNs for missing values where features don't have the same attributes
gencode_all = pd.concat(
    [gencode_genes, 
     gencode_exons_codons_utrs_cds, 
     gencode_transcripts_selenocyteines], 
     ignore_index=True
     ).reset_index().drop('index', axis=1)

# To confirm that all features are included in the new dataframe, uncomment the following line
# gencode_all.feature.unique()

In [23]:
gencode_genes.head()

,seqname,start,end,attribute,feature,strand,source,gene_name,gene_id,gene_type,gene_level
0,chr1,11869,14409,ID=ENSG00000290825.1;gene_id=ENSG00000290825.1...,gene,+,HAVANA,DDX11L2,ENSG00000290825.1,lncRNA,2
1,chr1,12010,13670,ID=ENSG00000223972.6;gene_id=ENSG00000223972.6...,gene,+,HAVANA,DDX11L1,ENSG00000223972.6,transcribed_unprocessed_pseudogene,2
2,chr1,14404,29570,ID=ENSG00000227232.5;gene_id=ENSG00000227232.5...,gene,-,HAVANA,WASH7P,ENSG00000227232.5,unprocessed_pseudogene,2
3,chr1,17369,17436,ID=ENSG00000278267.1;gene_id=ENSG00000278267.1...,gene,-,ENSEMBL,MIR6859-1,ENSG00000278267.1,miRNA,3
4,chr1,29554,31109,ID=ENSG00000243485.5;gene_id=ENSG00000243485.5...,gene,+,HAVANA,MIR1302-2HG,ENSG00000243485.5,lncRNA,2


In [10]:
list(gencode_all.columns)

['seqname',
 'start',
 'end',
 'attribute',
 'feature',
 'strand',
 'source',
 'gene_name',
 'gene_id',
 'gene_type',
 'gene_level',
 'exon_id',
 'exon_number',
 'transcript_name',
 'transcript_type']

## Saving, sorting, and indexing new annotation reference files

The gene annotation files will be saved in bed format with tab delimition and no header row. The rows will be sorted by chromosome, start, and end (in that order).

`gencode_all` contains all feature types and will be saved  as `gencode.v44.annotation.all.sorted.formatted.bed` with the columns:
    ```['seqname', 'start', 'end', 'attribute',
    'feature', 'strand', 'source', 'gene_name',
    'gene_id', 'gene_type', 'gene_level', 'exon_id',
    'exon_number', 'transcript_name', 'transcript_type']```

`gencode_gene` contains genes only and will be saved  as `gencode.v44.annotation.genes.sorted.formatted.bed` with the columns:
    ```['seqname', 'start', 'end', 'attribute',
    'feature', 'strand', 'source', 'gene_name',
    'gene_id', 'gene_type', 'gene_level']```

In [26]:
#### Save the gencode dataframes into files to be sorted and converted to bed files
gencode_all.to_csv('gencode.v44.annotation.all.tsv', index=False, header = False, sep="\t")
gencode_genes.to_csv('gencode.v44.annotation.genes.tsv', index=False, header = False, sep="\t")

In [27]:
%%bash -s gencode.v44.annotation.all.tsv
# Sort by chromosome, start, and end
cut -f 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15 $1 | sort -k1,1 -k2,2n -k3,3n > gencode.v44.annotation.all.sorted.formatted.bed

# Compress the bed file
bgzip -f gencode.v44.annotation.all.sorted.formatted.bed

# Index the bed file
tabix -f -p bed gencode.v44.annotation.all.sorted.formatted.bed.gz

In [29]:
%%bash -s gencode.v44.annotation.genes.tsv
# Sort by chromosome, start, and end
cut -f 1,2,3,4,5,6,7,8,9,10,11 $1 | sort -k1,1 -k2,2n -k3,3n > gencode.v44.annotation.genes.sorted.formatted.bed

# Compress the bed file
bgzip -f gencode.v44.annotation.genes.sorted.formatted.bed

# Index the bed file
tabix -f -p bed gencode.v44.annotation.genes.sorted.formatted.bed.gz